In [10]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [11]:
df_labeled = pd.read_csv("data_label_manual.csv")

df_unlabeled = pd.read_csv("sisa_data.csv")

print("Data berlabel:")
display(df_labeled.head())

print("Data tanpa label:")
display(df_unlabeled.head())


Data berlabel:


,clean_text,sentimen
0,yes betul sebagai pelatih timnas menangani tim...,positif
1,gua gak peduli siapa yang main yang penting ti...,positif
2,peluang timnas indonesia ke piala dunia masih ...,positif
3,berharap menang dari jepang dianggap gak masuk...,positif
4,iya iya gak boleh ngarep menang dari jepang ga...,positif


Data tanpa label:


,clean_text
0,peringkat timnas indonesia di ranking fifa dis...
1,pemain timnas indonesia jalani latihan terpisa...
2,timnas indonesia masih berjuang shin taeyong d...
3,timnas indonesia u tancap gas jelang laga uji ...
4,terima tawaran uji coba lawan timnas u indones...


In [12]:
text_col_labeled = df_labeled.columns[0]
label_col = df_labeled.columns[1]

text_col_unlabeled = df_unlabeled.columns[0]

X_labeled = df_labeled[text_col_labeled]
y_labeled = df_labeled[label_col]

X_unlabeled = df_unlabeled[text_col_unlabeled]


In [13]:
model = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=5000,
        ngram_range=(1,2)
    )),
    ("clf", MultinomialNB())
])


In [14]:
confidence_threshold = 0.9
max_iter = 10

for i in range(max_iter):
    print(f"\n🔁 Iterasi ke-{i+1}")

    # Latih model
    model.fit(X_labeled, y_labeled)

    # Prediksi probabilitas
    probs = model.predict_proba(X_unlabeled)
    max_probs = probs.max(axis=1)
    pseudo_labels = model.predict(X_unlabeled)

    # Ambil data dengan confidence tinggi
    confident_idx = np.where(max_probs >= confidence_threshold)[0]
    print(f"Data confident ditemukan: {len(confident_idx)}")

    if len(confident_idx) == 0:
        print("⛔ Tidak ada data confident, proses dihentikan")
        break

    # Tambahkan ke data berlabel
    X_labeled = pd.concat([X_labeled, X_unlabeled.iloc[confident_idx]])
    y_labeled = pd.concat([y_labeled, pd.Series(pseudo_labels[confident_idx])])

    # Hapus dari data unlabeled
    X_unlabeled = X_unlabeled.drop(
        X_unlabeled.index[confident_idx]
    ).reset_index(drop=True)



🔁 Iterasi ke-1
Data confident ditemukan: 0
⛔ Tidak ada data confident, proses dihentikan


In [15]:
df_unlabeled["label_prediksi"] = model.predict(df_unlabeled[text_col_unlabeled])

display(df_unlabeled.head())


,clean_text,label_prediksi
0,peringkat timnas indonesia di ranking fifa dis...,positif
1,pemain timnas indonesia jalani latihan terpisa...,negatif
2,timnas indonesia masih berjuang shin taeyong d...,negatif
3,timnas indonesia u tancap gas jelang laga uji ...,positif
4,terima tawaran uji coba lawan timnas u indones...,negatif


In [16]:
df_unlabeled.to_csv("hasil_labeling_self_training.csv", index=False)

print("✅ hasil_labeling_self_training.csv berhasil dibuat")


✅ hasil_labeling_self_training.csv berhasil dibuat
